# More Pandas

![more_pandas](https://media.giphy.com/media/H0Qi5W2KzU5UI/giphy.gif)

### Scenario
You have decided that you want to start your own animal shelter, but you want to get an idea of what that will entail and to get more information about planning. In this lecture, we'll look at a real data set collected by Austin Animal Center over several years and use our pandas skills from the last lecture and learn some new ones in order to explore these data further.




#### Our goals in this notebook are to be able to: <br/>

- Apply and use `.map()` and `.applymap()` from the Pandas library
- Review lambda functions and use them in coordination with above functions
- Explain what a groupby object is and split a DataFrame using `.groupby()`
- Change the dtype of a column to datetime and interact with a datetime object

#### Getting started

Let's take a moment to download and to examine the [Austin Animal Center data set](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Outcomes/9t4d-g238/data). 

Let's take a look at the data:

In [ ]:
import numpy as np
import pandas as pd
import requests

url = 'https://data.austintexas.gov/resource/9t4d-g238.json'
response = requests.get(url)
animals = pd.DataFrame(response.json())
animals.head()

![hive mind](https://media.giphy.com/media/l0MYttFGk98Y4e4h2/giphy.gif)


What kinds of questions can we ask these data and what kinds of information can we get back?
Start filling in the [group question doc](https://docs.google.com/document/d/15VFVdzx1-oFHuzal9xcNvXgr9QMtwbqIzpIzpzfzhFM/edit) together.  You can either add an individual question, or contribute to filling out another students question.  

# Quick Exploration

In [ ]:
animals.info()

In [ ]:
animals.describe()

In [ ]:
# Let's apply a bit of what we learned on Friday and investigate na's
animals.isna().sum()

In [ ]:
animals['name'] = animals['name'].fillna('unnamed')

In [ ]:
animals.fillna('no_type_or_subtype', inplace=True)

In [ ]:
animals.isna().sum()

### 1. Applying and using map and applymap from the Pandas library

The built in **map** operator takes a function and applies it to every element of an iterable

Map is an example of Python's implementation of [functional programming](https://docs.python.org/3/howto/functional.html), which we won't spend much lecture time on, except a brief description of the difference between OOP and functional.  In OOP, objects have changing state.  A DataFrame object has an attribute shape which outputs the number of rows and columns.  Dropping rows changes the underlying object and its attribute: the shape attribute has changed to reflect the reduced number of rows.  Functional programming relies on functions (like we've learned to write in Python) which take inputs and produce outputs, instead of relying on changing state of objects.  Some functional programming languages are Haskell and Lisp. 

In [ ]:
def divisible_by_5(number):
    
    '''
    Parameter: an integer
    return numbers divisible by five
    '''
    
    if number % 5 == 0:
        return True
    else:
        return False

numbers = [17,29,30045, 125]

list(map(divisible_by_5, numbers))

The Pandas library has several similar methods associated with Dataframes and Series. Let's explore them.

# DataFrame.applymap(),  Series.apply(), and DataFrame.apply()

## DataFrame.applymap()
The ```.applymap()``` method takes a function as input that it will then apply to every entry in the dataframe.

In [ ]:
def long_string(string):
    '''
    Parameter: a string
    returns: a boolean denoting whether it is 
    longer than 10 characters
    '''
    
    if len(string) > 10:
        return True
    else:
        return False

In [ ]:

animals_strings = animals.applymap(str)
animals_strings.applymap(long_string)

The **.map()** method takes a function as input that it will then apply to every entry in the Series.

Let's write a function to consolodate sex_upon_outcome to male, female, or unknown   

First, explore the unique values:

In [ ]:
animals['sex_upon_outcome'].unique()

In [ ]:
# we could also us np.unique() with the return_counts parameter

np.unique(animals['sex_upon_outcome'], return_counts=True)


In [84]:
def male_or_female(value_from_series):
    
    """
    This is a docstring...
    
    Parameter: a value from the age_upon_outcome series
    in the Austin Animal Shelter dataset.  
    There are five possible values:
    Spayed Female', 'Unknown', 'Intact Female', 'Intact Male',
       'Neutered Male'
       
    Returns:
    female, male, unknown
    
    """
    
    if 'female' in value_from_series.lower():
        return 'female'
    
    #Add a space before male to ensure that female is not included
    elif ' male' in value_from_series.lower():
        return 'male'
    
    else:
        return 'unknown'
    
    
animals['sex_upon_outcome'].apply(male_or_female)   

0      female
1      female
2      female
3      female
4        male
        ...  
995    female
996    female
997      male
998      male
999    female
Name: sex_upon_outcome, Length: 1000, dtype: object

In [85]:
# We can now make a new column, sex:
    
animals['sex'] = animals['sex_upon_outcome'].apply(male_or_female)
animals.head()

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,outcome_subtype,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,days_upon_outcome,sex
0,A817445,Girlie,2020-05-17T13:53:00.000,2020-05-17T13:53:00.000,2011-05-16,Transfer,Partner,Dog,Spayed Female,9 years,Chihuahua Shorthair,Brown,3285.0,female
1,A817061,*Nancy Jo,2020-05-17T13:52:00.000,2020-05-17T13:52:00.000,2019-08-07,Adoption,no_type_or_subtype,Dog,Spayed Female,9 months,German Shepherd Mix,Black/Brown,3285.0,female
2,A817198,*Moon,2020-05-17T13:04:00.000,2020-05-17T13:04:00.000,2019-05-09,Died,In Foster,Cat,Intact Female,1 year,Siamese,Blue Point,365.0,female
3,A817154,*Marinara,2020-05-17T12:21:00.000,2020-05-17T12:21:00.000,2019-11-24,Adoption,no_type_or_subtype,Dog,Spayed Female,5 months,Australian Cattle Dog/Labrador Retriever,Black Brindle,1825.0,female
4,A816636,Sugar,2020-05-17T12:03:00.000,2020-05-17T12:03:00.000,2019-05-14,Rto-Adopt,no_type_or_subtype,Cat,Neutered Male,1 year,Domestic Shorthair,Brown Tabby,365.0,male


Now let's have some fun.  Let's convert age upon outcome to days, using map():

In [ ]:
# First, checkout what happens when we split on a space

list(animals['age_upon_outcome'].str.split(' '))

# Pair program #1: 
Take 5 minutes to fill in the function below with code that converts age upon outcome to days upon outcome.

In [ ]:
# check what values we have for time frame
unit_values = [age[0] if age[0] == 'NULL' 
               else age[1] for age in 
               animals['age_upon_outcome'].str.split(' ')]
set(unit_values)

Now, fill in the definition below to convert the ages to days

In [ ]:

def age_to_days(age):
    
    '''
    params: age upon outcome of shelter animal. 
    A number followed by a unit of time 
    'NULL', 'days', 'month', 'months', 'week', 'weeks', 'year', 'years'
    
    returns: days old at outcome
    '''
    
    age_split = age.split(' ')
    
    if len(age_split) == 1:
        return 'NULL'
    
    elif ... :
        return
    
    elif ... :
         pass
    
    elif ... :
         pass
    
    else:
         pass
    
    
animals['age_upon_outcome'].map(age_to_days)


In [ ]:
#__SOLUTION__
def age_to_days(age):
    
    '''
    params: age upon outcome of shelter animal. 
    A number followed by a unit of time 
    'NULL', 'days', 'month', 'months', 'week', 'weeks', 'year', 'years'
    
    returns: days old at outcome
    '''
    
    age_split = age.split(' ')
    
    if len(age_split) == 1:
        return np.nan
    
    elif age_split[1] == 'days' :
        return int(age_split[0])
    
    elif age_split[1] in (['month' or 'months']):
        return int(age_split[0]) * 30
    
    elif age_split[1] in ['week' or 'weeks'] :
        return int(age_split[0]) * 7
    
    else:
        return int(age_split[0]) * 365
    
    
animals['days_upon_outcome'] = animals['age_upon_outcome'].map(age_to_days)


# DataFrame.apply()

In [96]:
animals.head()

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,outcome_subtype,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,days_upon_outcome,sex
0,A817445,Girlie,2020-05-17T13:53:00.000,2020-05-17T13:53:00.000,2011-05-16,Transfer,Partner,Dog,Spayed Female,9 years,Chihuahua Shorthair,Brown,3285.0,female
1,A817061,*Nancy Jo,2020-05-17T13:52:00.000,2020-05-17T13:52:00.000,2019-08-07,Adoption,no_type_or_subtype,Dog,Spayed Female,9 months,German Shepherd Mix,Black/Brown,3285.0,female
2,A817198,*Moon,2020-05-17T13:04:00.000,2020-05-17T13:04:00.000,2019-05-09,Died,In Foster,Cat,Intact Female,1 year,Siamese,Blue Point,365.0,female
3,A817154,*Marinara,2020-05-17T12:21:00.000,2020-05-17T12:21:00.000,2019-11-24,Adoption,no_type_or_subtype,Dog,Spayed Female,5 months,Australian Cattle Dog/Labrador Retriever,Black Brindle,1825.0,female
4,A816636,Sugar,2020-05-17T12:03:00.000,2020-05-17T12:03:00.000,2019-05-14,Rto-Adopt,no_type_or_subtype,Cat,Neutered Male,1 year,Domestic Shorthair,Brown Tabby,365.0,male


In [92]:
dog_days_std = animals.groupby('animal_type').std().loc['Dog']
dog_days_mean = animals.groupby('animal_type').mean().loc['Dog']
dog_days_mean

days_upon_outcome    1315.363484
Name: Dog, dtype: float64

In [145]:
dog_days_mean = animals.groupby('animal_type').mean().loc['Dog']
dog_days_std = animals.groupby('animal_type').std().loc['Dog']
(animals.days_upon_outcome > int(dog_days_mean + dog_days_std*2)).value_counts()

False    927
True      73
Name: days_upon_outcome, dtype: int64

In [157]:
# let's make a boolean column that crease a boolean for old dogs that get adopted 

def old_dogs_adopted(row):

    '''
    Parameter: Row from the Austin Animal Shelter
    Returns: Boolean signifying records of old dogs that were adopted
    '''
    
    if (
                row['animal_type'] == 'Dog')\
                and (row['outcome_type'] =='Adoption')\
                and (row['days_upon_outcome'] > int(dog_days_mean + dog_days_std*2)):
        return True

    else:
        return False

animals['old_adopted_dogs'] = animals.apply(old_dogs_adopted, axis = 1)

In [159]:
animals[animals.old_adopted_dogs == True]

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,outcome_subtype,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,days_upon_outcome,sex,old_adopted_dogs
21,A815992,*Sadie,2020-05-16T13:18:00.000,2020-05-16T13:18:00.000,2010-04-02,Adoption,Foster,Dog,Spayed Female,10 years,Labrador Retriever/German Shepherd,Tan,3650.0,female,True
35,A817148,*Pickle,2020-05-15T12:58:00.000,2020-05-15T12:58:00.000,2019-07-08,Adoption,no_type_or_subtype,Dog,Neutered Male,10 months,German Shepherd,Brown/Black,3650.0,male,True
55,A816641,Max,2020-05-14T09:34:00.000,2020-05-14T09:34:00.000,2019-05-23,Adoption,Foster,Dog,Neutered Male,11 months,German Shepherd Mix,Sable,4015.0,male,True
67,A816783,*Fern,2020-05-13T12:47:00.000,2020-05-13T12:47:00.000,2019-05-27,Adoption,no_type_or_subtype,Dog,Spayed Female,11 months,Pit Bull,Tan/White,4015.0,female,True
191,A815198,*Donut,2020-05-05T09:02:00.000,2020-05-05T09:02:00.000,2010-04-06,Adoption,Foster,Dog,Spayed Female,10 years,Rat Terrier Mix,Tricolor,3650.0,female,True
310,A643015,Mojo,2020-04-24T15:30:00.000,2020-04-24T15:30:00.000,2008-12-03,Adoption,Foster,Dog,Neutered Male,11 years,Boxer,Brown/White,4015.0,male,True
311,A796805,*Etta,2020-04-24T14:34:00.000,2020-04-24T14:34:00.000,2009-06-05,Adoption,Foster,Dog,Spayed Female,10 years,Labrador Retriever,Tan/White,3650.0,female,True
330,A416147,Champion,2020-04-21T14:28:00.000,2020-04-21T14:28:00.000,2005-11-03,Adoption,Foster,Dog,Neutered Male,14 years,Labrador Retriever Mix,Black/White,5110.0,male,True
357,A811827,*Bo,2020-04-20T17:02:00.000,2020-04-20T17:02:00.000,2019-06-10,Adoption,Foster,Dog,Neutered Male,10 months,German Shepherd Mix,Tan,3650.0,male,True
400,A809473,Roscoe,2020-04-16T10:46:00.000,2020-04-16T10:46:00.000,2009-11-25,Adoption,Foster,Dog,Neutered Male,10 years,Labrador Retriever,Chocolate,3650.0,male,True


### DataFrame.apply()
DataFrame.apply() takes a function as a parameter, and applies it every element of an axis.  It is especially useful if we want to use logic that compares multiple column values.

In [118]:

nimal Shelter dataset
    Returns; A boolean denoting if the record is an adopted old dog.
    '''

    if row['animal_type'] == 'Dog'  :
        return True
    else:
        return False

animals.apply(old_dogs_adopted, axis=1).value_counts()


True     597
False    403
dtype: int64

### Anonymous Functions (Lambda Abstraction)

Simple functions can be defined right in the function call. This is called 'lambda abstraction'; the function thus defined has no name and hence is "anonymous".

# Student Screen Share (without answer directly below)
Use another lambda function to convert days days upon outcome to weeks upon outcome <br>


In [ ]:
# your code here

In [ ]:
#__SOLUTION__
animals['days_upon_outcome'].map(lambda x: x//7)

### 2. Methods for Re-Organizing DataFrames: .groupby()

Those of you familiar with SQL have probably used the GROUP BY command. (And if you haven't, you'll see it very soon!) Pandas has this, too.

The .groupby() method is especially useful for aggregate functions applied to the data grouped in particular ways.

In [ ]:
animals.groupby('animal_type')

In [ ]:
animals.columns

We can group by multiple columns, and also return a DataFrameGroupBy object

Notice the object type [DataFrameGroupBy](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) object. 

#### .groups and .get_group()

In [ ]:
animals.groupby(['animal_type', 'outcome_type'])

In [ ]:
# This retuns each group indexed by the group name: I.E. 'Bird', along with the row indices of each value
animals.groupby('animal_type').groups

Once we know we are working with a type of object, it opens up a suite of attributes and methods. One attribute we can look at is groups.

In [ ]:
animals.groupby('animal_type').get_group('Dog')

In [ ]:
# Once we know the group indices, we can return the groups using those indices.

#### Aggregating

In [ ]:
# Same goes for multi index groupbys
animal_outcome = animals.groupby(['animal_type', 'outcome_type'])
animal_outcome.groups


In [ ]:
# animal_outcome.groups is a dictionary, so we can access the group names using keys()
animal_outcome.groups.keys()

In [ ]:
# We can then get a specific group, such as Cats that were adopted
animal_outcome.get_group(('Cat', 'Adoption'))

In [ ]:
# Other methods
animal_outcome.first()

In [ ]:
animal_outcome.last()

In [ ]:
animals.groupby('animal_type').mean()

Once again, as we will see in SQL, in order to return a groupby objects are intended to be used with aggregation. In SQL, we will see that our queries that include GROUP BY require aggregation performed on columns.

We can use sum, mean, count, max, min... Find a list of common aggregations [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html)

In [ ]:
# Group by multiple indices
animals.groupby(['animal_type', 'outcome_type']).get_group(('Cat', 'Transfer')).describe()